함수호출

In [1]:
import pandas as pd
import time,datetime
import numpy as np
from pandas import *
from informs_Copy2 import portfolio
import class_informs_Copy3 as inform

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import math
import random as ran
from scipy.stats import truncnorm
#import seaborn as sbn

import time,datetime
import cmath


import re
import matplotlib.pyplot as plt

import os
import ipyparallel as ipp

import generator
import discriminator

import operator


데이터 전처리

In [2]:
#timeseries data
all_data=pd.read_csv('data/Timeseries_data_SP500.csv')

all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

date_list=list(set(list(all_data['date'])))
date_list.sort()

dic_data = {k: v for k, v in all_data.groupby('date')}

results_=pd.read_csv('data/results_template.csv')
results_=results_.rename(columns={'DATE':'date','SEDOL':'sedol','WEIGHTS':'w','FOUR_WEEKLY_RETURN':'r'})
results_['date']=pd.to_datetime(results_['date'])
dic_results_={k: v for k, v in results_.groupby('date')}

dic_r_all={}
for i in date_list:
    rr={}
    for j,k in zip(list(dic_results_[i]['sedol']),list(dic_results_[i]['r'])):
        rr[j]=k
    dic_r_all[i]=rr



def preprocessing(date):
    
    using_dic = dic_data[pd.to_datetime(date)]
    using_dic['index']=list(using_dic.index)

    asset_list = []
    for i in using_dic["sedol"]:
        asset_list.append(i)


    dic_sedol_as = {using_dic["sedol"][i] : using_dic["as"][i] for i in using_dic.index}

    dic_bench = {using_dic["sedol"][i] : using_dic["bw"][i] for i in using_dic.index}

    dic_beta = {using_dic["sedol"][i] : using_dic["beta"][i] for i in using_dic.index}

    dic_sector = {using_dic["sector"][i] : [] for i in using_dic.index }

    dic_MCAP = {using_dic["mq"][i] : [] for i in using_dic.index}

    dic_aseet_MCAP = {using_dic["sedol"][i] : using_dic["mq"][i] for i in using_dic.index}
    
    dic_aseet_sector = {using_dic["sedol"][i] : using_dic["sector"][i] for i in using_dic.index}
    
    for i in using_dic.index:
        dic_sector[using_dic["sector"][i]].append(using_dic["sedol"][i])
        dic_MCAP[using_dic["mq"][i]].append(using_dic["sedol"][i])



    #risk:cov_mat
    date_str=str(date)[:10]
    risk_data=pd.read_csv('data/Riskmodels/cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    #         return risk_data,risk_sedol,risk_mat

    
    alpha = []
    
    dic_risk_index = {}
    index_ri = 0

    for i in risk_sedol:
        dic_risk_index.update({i:index_ri})
        index_ri += 1
        alpha.append(-1*dic_sedol_as[i])

    dic_r={dic_results_[date]["sedol"][i] : dic_results_[date]["r"][i] for i in using_dic.index}

    return using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector,dic_risk_index

print("complete")



complete


In [21]:
date=date_list[1]
using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector,dic_risk_index=preprocessing(date)
print(len(dic_r))
len(risk_sedol)

493


493

함수 정의

In [3]:
def cal_obj1(sol, risk_mat, as_) :

    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, as_*10000)
    obj = s_r-s_a
    return obj

def cal_obj(sol, risk_mat, as_) :

    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    sol_d = list(np.array(sol) - np.array(using_dic['bw']))
    s_rr = np.dot(sol_d, risk_mat)
    s_r = np.dot(s_rr, sol_d)
    s_a = np.dot(sol_d, as_*10000)
    obj = s_r-s_a
    return obj



def select_candi(sol) :
    
    diff_sub = np.array([0]*len(as_), float)
    diff_add = np.array([0]*len(as_), float)
    candi = []
    
#     if sum(sol) >= 50 :
#         for val in range(len(sol)) :
#             if sol[val] > 0. :
#                 candi.append(val)
        
#     else :

    for i in range(len(sol)) :
        if(sol[i] > 0) :
            w = sol[i]
            sol[i] = 0
            diff_sub[i] = cal_obj(sol, risk_mat, as_)
            if min(diff_sub[diff_sub>0]) == diff_sub[i] :
                for j in range(len(sol)) :
                    if sol[j] == 0 :
                        sol[j] = w
                        diff_add[j] = cal_obj(sol, risk_mat, as_)
                        sol[j] = 0
            sol[i] = w
        
    sol[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    sol[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]] = 0.1

    for val in range(len(sol)) :
        if sol[val] > 0. :
            candi.append(val)
    print(candi)
    print("candi")

    return candi



def update(sol,lim_time,as_,w_pre,O_scale) :
    start_time = time.time()
    
    
    
    
    ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)
    ex.set_omega(risk_mat)

    new_list = []

    con = []
    nums = 0
    
    w_pre_len = 0
    sort_w_p = sorted(w_pre.items(),key=operator.itemgetter(1))
    save_w_p = len(sort_w_p) - 1
    print(w_pre[sort_w_p[save_w_p][0]],"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    w_p_key_list = []
    sol_len = 0
    
    for l in w_pre.keys():
        if w_pre[l] > 0 :
            w_pre_len += 1
            w_p_key_list.append(l)
            
    for k in range(int(w_pre_len/10)):
        if sol[sort_w_p[save_w_p][0]] < 0.9:
            sol.update({sort_w_p[save_w_p][0]:1})
        save_w_p = save_w_p - 1
    
    for k in risk_sedol : 
        if sol[k] > 0:
            sol_len += 1
    
    out_list = []

    if sol_len > 70:
        while(sol_len > 70):
            out_list = []
            for k in risk_sedol :
                if sol[k] == 1 and k not in w_p_key_list:
                    out_list.append(k)

            out_risk_key = 10000
            out_risk_val = -10000
            for m in out_list:
                if out_risk_val < risk_mat[dic_risk_index[m]][dic_risk_index[m]]:
                    out_risk_key = m
                    out_risk_val = risk_mat[dic_risk_index[m]][dic_risk_index[m]]
            sol[out_risk_key] = 0
            sol_len = sol_len - 1
            
            
        for k in risk_sedol :
            new_list.append(sol[k])
            if sol[k] == 1:
                con.append(nums)
            nums += 1
    else:
        for k in risk_sedol :
            new_list.append(sol[k])
            if sol[k] == 1:
                con.append(nums)

            nums += 1
    
    
    result_list = []
    result_list2 = []
    
    
    
    print(con)
    print(len(con))
    print("con")
    while(True):
#         opt_return = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,
#         MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,multiple=10000,time_init=90,cad_set = sol)[0][2]
#         opt_return_list.append(opt_return)
        
        sol_len = 0
        for k in risk_sedol :
            if sol[k] > 0:
                sol_len += 1

        if time.time() - start_time > lim_time:
            break
        
        ex.set_con(cons=con)

            
                
                
        new_list2 = []
        new_list3 = []
        try:
            aabbaa = ex.solve(0.001)
            
            
            if aabbaa[0] == 0:
                print("TE infeasible")
                TE_out = aabbaa[1]
                print("TE_out : ",TE_out)
                con = TE_change(sol,TE_out)
                sol = {}
                cheknumb = 0
                connum = 0
                for i in risk_sedol:
                    if connum < len(con):
                        if con[connum] == cheknumb:
                            sol.update({i:1})
                            connum += 1
                        else:
                            sol.update({i:0})
                    else:
                            sol.update({i:0})
                    cheknumb += 1
                
            else:
            
                if aabbaa[0] == "sector":
                    print("S infeasible")
                    con = Sector_change(sol,aabbaa[1],aabbaa[2])
                    cheknumb = 0
                    connum = 0
                    sol = {}
                    for i in risk_sedol:
                        if connum < len(con):
                            if con[connum] == cheknumb:
                                sol.update({i:1})
                                connum += 1
                            else:
                                sol.update({i:0})
                        else:
                                sol.update({i:0})
                        cheknumb += 1

                elif aabbaa[0] == "MCAPQ":
                    print("M infeasible")
                    con = MCAPQ_change(sol,aabbaa[1],aabbaa[2])
                    cheknumb = 0
                    connum = 0
                    sol = {}
                    for i in risk_sedol:
                        if connum < len(con):
                            if con[connum] == cheknumb:
                                sol.update({i:1})
                                connum += 1
                            else:
                                sol.update({i:0})
                        else:
                                sol.update({i:0})
                        cheknumb += 1

                else:
                    aaaa = aabbaa[0][0]
                    bbbb= aabbaa[0][1]
                    for k in risk_sedol :
                        new_list2.append(aaaa[k])
                    for k in risk_sedol :
                        new_list3.append(bbbb[k])
                    new_list = new_list2

                    result_list.append(aabbaa[0][2])
                    result_list2.append(aabbaa[0][0])
                    print("obj_val")
                    print(cal_obj1(new_list3, risk_mat, as_))
                    TE_out = aabbaa[1]
                    con = TE_change(sol)
                    cheknumb = 0
                    connum = 0
                    sol = {}
                    for i in risk_sedol:
                        if connum < len(con):
                            if con[connum] == cheknumb:
                                sol.update({i:1})
                                connum += 1
                            else:
                                sol.update({i:0})
                        else:
                                sol.update({i:0})
                        cheknumb += 1
                    print("cplex_val")
                    print(aabbaa[0][2])
                

        except:
            print("cplex no solution")
            sc = Sector_check(sol)
            if sc == 100:
                mc = MCAPQ_check(sol)
                if mc == 100:
                    con = TE_change(sol)
                else:
                    con = MCAPQ_change(sol,mc[0],mc[1])                    
            else:
                con = Sector_change(sol,sc[0],sc[1])
                
            
            cheknumb = 0
            connum = 0
            sol = {}
            for i in risk_sedol:
                if connum < len(con):
                    if con[connum] == cheknumb:
                        sol.update({i:1})
                        connum += 1
                    else:
                        sol.update({i:0})
                else:
                        sol.update({i:0})
                cheknumb += 1



    return result_list,result_list2

def update2(sol,lim_time,as_,w_pre,O_scale) :
    start_time = time.time()
    
    
    
    
    ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)
    ex.set_omega(risk_mat)

    new_list = []

    con = []
    nums = 0
    
        
    
    
    
    for k in risk_sedol :
        new_list.append(sol[k])
        if sol[k] == 1:
            con.append(nums)

        nums += 1
    result_list = []
    result_list2 = []
    
    
    
    print(con)
    print("con")
    while(True):

        if time.time() - start_time > lim_time:
            break
        
        ex.set_con(cons=con)
        new_list2 = []
        new_list3 = []

        aabbaa = ex.solve(0.01)


        if aabbaa != 0:
            aaaa = aabbaa[0]
            bbbb= aabbaa[1]
            for k in risk_sedol :
                new_list2.append(aaaa[k])
            for k in risk_sedol :
                new_list3.append(bbbb[k])
            new_list = new_list2

            result_list.append(aabbaa[2])
            result_list2.append(aabbaa[0])
            con = select_candi(new_list)


        else:
            print("TE infeasible")
            con = select_candi(new_list)





    return result_list,result_list2



def GAN(rotation):
    X_dim = nbasset
    HL_dim = 50
    Z_dim = 10

    np.random.seed()
#     G = Generator(Z_dim, HL_dim, X_dim)

#     sample_D = Discriminator(X_dim, train=False)
#     train_D = Discriminator(X_dim, train=True)

    w_pre_gan=np.zeros(len(asset_list))

    for i,j in enumerate(asset_list):
        if j in w_pre.keys(): w_pre_gan[i]=w_pre[j]
        else:w_pre_gan[i]=0.
            
    car=50

    G = generator.Generator(Z_dim, HL_dim, X_dim)

    
    sample_D = discriminator.Discriminator(in_dim=X_dim, train=False, risk_mat=risk_mat, bw=bw, beta=beta, alpha=alpha, w_11=0.1
                                           ,w_pre_gan=w_pre_gan,MONTH=n,c_lower=car)
    train_D = discriminator.Discriminator(in_dim=X_dim, train=True, risk_mat=risk_mat, bw=bw, beta=beta, alpha=alpha, w_11=0.1
                                          ,w_pre_gan=w_pre_gan,MONTH=n,c_lower=car)
    G_solver = optim.Adam(G.parameters(), lr=2e-3)

    mb_size = 128

    best_obj = 100000.0
    best_sols = []

    losses = []

    start_time = time.time()

    feasible_threshold = 0.95

    adaptive_inf_scale = float(nbasset)

    for it in range(rotation):
        if it % 900 == 0:
            sample_noise = generator.sample_Z(128, Z_dim)
            samples = G(sample_noise)

            sample_D.Zstar = best_obj
            sample_D.inf_scale = adaptive_inf_scale

            sample_D_values = sample_D(samples).data.numpy()
            sample_obj_values = sample_D.objs.data.numpy()

            fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
            sols = samples.data.numpy()[sample_D_values > feasible_threshold]

            if len(fea_sample_obj_values > 0):
                best_idx = np.argmin(fea_sample_obj_values)
                if np.min(fea_sample_obj_values) < best_obj:
                    best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                    best_sols.append(sols[best_idx])

        z = Variable(torch.randn(mb_size, Z_dim))

        train_D.Zstar = best_obj
        train_D.inf_scale = adaptive_inf_scale

        G_loss = -torch.mean(torch.log(train_D(G(z))))

        G.zero_grad()
        G_loss.backward()
        G_solver.step()
        
    sam=[]
    for i in range(50):
        n = sample_Z(128, 10)
        samples = G(n)
        sam.append(samples)
        
    return best_sols, sols,fea_sample_obj_values,sam

        

def GAN_reulst(best_sols, sols,fea_sample_obj_values):    
    good_sols=np.argsort(fea_sample_obj_values)[:10]
    card_list=[list(np.where(sols[i]>0.001)[0]) for i in range(len(good_sols))]
    card_freq=np.zeros(nbasset)
    for i in range(len(card_list)):
        for j in card_list[i]:
            card_freq[j]+=1
    best=np.argsort(sum(sols))[::-1][:len(card_freq[card_freq>len(card_list)-5])]
    bestsol=np.where(best_sols[0]>0.001)[0]
    return best,bestsol

def init_cplex(risk_mat,risk_sedol):
    sedol_var_list =[]
    for i in risk_sedol:
        sedol_var_list.append("d"+str(i))

    for i in risk_sedol:
        sedol_var_list.append("q"+str(i))

    sedol_var_list.append("assum")
    #global alpha
    alpha = []
    for i in risk_sedol:
        alpha.append(-10000.*dic_sedol_as[i])

    qmat=[]
    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(20000*j)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(1):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    q_con1 = []
    q_con2 = []
    q_val = []


    for i in range(len((risk_mat[0]))):
        for j in range(len(risk_mat[0])):
            if j >= i:
                q_con1.append(i)
                q_con2.append(j)
                if i == j:
                    ex_list = list(risk_mat[i])
                    q_val.append(0.5*ex_list[j]*10000)
                else:
                    ex_list = list(risk_mat[i])
                    q_val.append(ex_list[j]*10000)

    Q_con = []
    Q_con.append(q_con1)
    Q_con.append(q_con2)
    Q_con.append(q_val)
    
    return Q_con, qmat, alpha

print("complete")

complete


In [4]:
def TE_change2(w_dic,TE_out):
    
    TE_in = -999
    CH_be = -99999
    nnn = 0
    candi = []
    
    
    
    for i in w_dic:
        if w_dic[i] == 0:
            if CH_be < abs(dic_bench[i]*dic_bench[i]*risk_mat[nnn][nnn]):
                CH_be = abs(dic_bench[i]*dic_bench[i]*risk_mat[nnn][nnn])
                TE_in = nnn
        nnn += 1
        
    w_list = []
    
    for i in risk_sedol:
        w_list.append(w_dic[i])
    
    
    w_list[TE_out] = 0
    w_list[TE_in] = 0.1

    for val in range(len(w_list)) :
        if w_list[val] > 0. :
            candi.append(val)
    print(candi)
    #print("candi")

    return candi



def TE_change(sol):
    w_list = []
    
    for i in risk_sedol:
        w_list.append(sol[i])


    
    diff_out = np.array([0]*len(as_), float)
    diff_sub = np.array([0]*len(as_), float)
    

    candi = []
    indnum = 0
    indnum2 = 0
    indnum3 = 0
    indnum4 = 0

    chek_one = 0

    for i in risk_sedol :
        if sol[i] > 0:
            chek_one += 1
            save_risk = 0
            for j in risk_sedol :
                if sol[j] > 0:
                    save_risk += risk_mat[indnum][indnum2]                                
                indnum2 += 1
            save_risk += alpha[indnum]
            diff_out[indnum] = save_risk 
        indnum += 1
        indnum2 = 0
    

    
    w_list[np.where(diff_out == max(diff_out[diff_out>0]))[0][0]] = 0   
    
    print(np.where(diff_out == max(diff_out[diff_out>0]))[0][0])
    
    in_cady = np.where(diff_out == max(diff_out[diff_out>0]))[0][0]
    
    for i in risk_sedol :
        if sol[i] == 0:
            save_risk = 0            
            for j in risk_sedol :
                if sol[j] > 0:
                    save_risk += risk_mat[indnum3][indnum4]
                indnum4 += 1
            save_risk += alpha[indnum3]
            save_risk += risk_mat[indnum3][indnum3]
            diff_sub[indnum3] = save_risk                     
        indnum3 += 1
        indnum4 = 0


    
    print(np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0])


    w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0.1

    for val in range(len(w_list)) :
        if w_list[val] > 0. :
            candi.append(val)
    print(candi)
    #print("candi")

    return candi
# date=date_list[0]
# using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector=preprocessing(date)

# as_ = using_dic['as']
# sol = {}
# indnummm = 0

# for i in risk_sedol:
#     if indnummm < 70:
#         sol.update({i:1})
#     else:
#         sol.update({i:0})
#     indnummm += 1
    

# aaaaaaaaaaa = 0
    
# for j in range(10):
#     con = TE_change2(sol)
#     aaaaaaaaaaa += 1
#     cheknumb = 0
#     connum = 0
#     sol = {}
#     for i in risk_sedol:
#         if connum < len(con):
#             if con[connum] == cheknumb:
#                 sol.update({i:1})
#                 connum += 1
#             else:
#                 sol.update({i:0})
#         else:
#                 sol.update({i:0})
#         cheknumb += 1    
    
    

In [5]:
def MCAPQ_check(w_dic):

    MCAPQ_sum_dic = {}
    d_sum_dic = {}
    d_sum_dic2 = {}
    #date=date_list[0]
    #using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP=preprocessing(date)

    for i in dic_MCAP:
#         updatenum = -0.1
        d_sum_dic.update({i:0})
        d_sum_dic2.update({i:0})
#         for j in dic_sector[i]:
#             updatenum += dic_bench[j]
#         Sector_sum_dic.update({i:updatenum})

    #print(Sector_sum_dic)

    for i in risk_sedol:
        if w_dic[i] > 0:
            savenum = d_sum_dic[dic_aseet_MCAP[i]]
            savenum += 0.05
            d_sum_dic.update({dic_aseet_MCAP[i]:savenum})
            savenum = 0
        else:
            savenum = d_sum_dic[dic_aseet_MCAP[i]]
            savenum = savenum - dic_bench[i]
            d_sum_dic.update({dic_aseet_MCAP[i]:savenum})
            savenum = 0
    
    
              
                                

                            
                            
            
    in_MCAPQ = -999
    out_MCAPQ = -999
    for i in d_sum_dic:
        if d_sum_dic[i] < -0.01:
            in_MCAPQ = i
            print("IN_MCAPQ :",in_MCAPQ)
            print(d_sum_dic[i],"1111111111111111")
            print(d_sum_dic)
            out_MCAPQ = max(d_sum_dic.keys(), key=(lambda k: d_sum_dic[k]))
            print("out : ",out_MCAPQ) 
            return in_MCAPQ,out_MCAPQ
            break

    
    
        
        
        
    if in_MCAPQ == -999:
#         for i in risk_sedol:
#             if w_dic[i] > 0:
#                 savenum = d_sum_dic2[dic_aseet_MCAP[i]]
#                 if 0.01 - dic_bench[i] > -0.05:
#                     savenum = savenum + 0.01 - dic_bench[i]
#                     d_sum_dic2.update({dic_aseet_MCAP[i]:savenum})
#                     savenum = 0
#                 else:
#                     savenum = savenum -0.05
#                     d_sum_dic2.update({dic_aseet_MCAP[i]:savenum})
#                     savenum = 0
#             else:
#                 savenum = d_sum_dic2[dic_aseet_MCAP[i]]
#                 savenum = savenum - dic_bench[i]
#                 d_sum_dic2.update({dic_aseet_MCAP[i]:savenum})
#                 savenum = 0
        
#         for i in d_sum_dic2:
#             if d_sum_dic2[i] > 0.01:
#                 out_MCAPQ = i
#                 in_MCAPQ = min(d_sum_dic2.keys(), key=(lambda k: abs(d_sum_dic2[k])))
#                 print(d_sum_dic2[i],"22222222222222222222222")
#                 print("IN_MCAPQ : ",in_MCAPQ)
#                 print("out : ",out_MCAPQ)
#                 return in_MCAPQ,out_MCAPQ
#                 break
        print("nono MC")
        return 100   
#     if in_MCAPQ == -999:
        
    

def MCAPQ_change2(sol,inM,outM):
    w_list = []
    
    for i in risk_sedol:
        w_list.append(sol[i])


    print("inM : ",inM)
    print("outM : ",outM)
    
    diff_sub = np.zeros((len(as_),len(as_)))

    candi = []
    indnum = 0
    indnum2 = 0
#     risk[A][A] + risk[A][E] +risk[E][A]
    for i in risk_sedol :
        if sol[i] > 0 and dic_aseet_MCAP[i] == outM:
            for j in risk_sedol :
                if sol[j] == 0 and dic_aseet_MCAP[j] == inM:
                    diff_sub[indnum][indnum2] = risk_mat[indnum][indnum] + 2*risk_mat[indnum][indnum2] - alpha[indnum] + alpha[indnum2]
                    
                indnum2 += 1
        indnum += 1
        indnum2 = 0


    print(np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0])
    print(np.where(diff_sub == min(diff_sub[diff_sub>0]))[1][0])

    w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[1][0]] = 0.1

    for val in range(len(w_list)) :
        if w_list[val] > 0. :
            candi.append(val)
    print(candi)
    #print("candi")

    return candi


def MCAPQ_change(sol,inM,outM):
    w_list = []
    
    for i in risk_sedol:
        w_list.append(sol[i])


    print("inM : ",inM)
    print("outM : ",outM)
    
    diff_out = np.array([0]*len(as_), float)
    diff_sub = np.array([0]*len(as_), float)
    

    candi = []
    indnum = 0
    indnum2 = 0
    indnum3 = 0
    indnum4 = 0
#     risk[A][A] + risk[A][E] +risk[E][A]

    chek_one = 0

    for i in risk_sedol :
        if sol[i] > 0 and dic_aseet_MCAP[i] == outM:
            chek_one += 1
            save_risk = 0
            for j in risk_sedol :
                if sol[j] > 0 and dic_aseet_MCAP[j] == outM:
                    save_risk += risk_mat[indnum][indnum2]                                
                indnum2 += 1
            save_risk += alpha[indnum]
            diff_out[indnum] = save_risk 
        indnum += 1
        indnum2 = 0
    
    if chek_one == 1:
        in_cady = -99999999999999999999
    
    w_list[np.where(diff_out == max(diff_out[diff_out>0]))[0][0]] = 0   
    
    print(np.where(diff_out == max(diff_out[diff_out>0]))[0][0])
    
    in_cady = np.where(diff_out == max(diff_out[diff_out>0]))[0][0]
    
    for i in risk_sedol :
        if sol[i] == 0 and dic_aseet_MCAP[i] == inM:
            save_risk = 0            
            for j in risk_sedol :
                if sol[j] > 0 and dic_aseet_MCAP[j] == outM and indnum4 != in_cady:

                    save_risk += risk_mat[indnum3][indnum4]
                indnum4 += 1
            save_risk += alpha[indnum3]
            save_risk += risk_mat[indnum3][indnum3]
            diff_sub[indnum3] = save_risk                     
        indnum3 += 1
        indnum4 = 0


    
    print(np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0])


    w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0.1

    for val in range(len(w_list)) :
        if w_list[val] > 0. :
            candi.append(val)
    print(candi)
    #print("candi")

    return candi
# date=date_list[0]
# using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector=preprocessing(date)

# as_ = using_dic['as']
# sol = {}
# indnummm = 0

# for i in risk_sedol:
#     if indnummm < 70:
#         sol.update({i:1})
#     else:
#         sol.update({i:0})
#     indnummm += 1
    

# for j in range(10):
#     con = MCAPQ_change2(sol,2,1)
#     cheknumb = 0
#     connum = 0
#     sol = {}
#     for i in risk_sedol:
#         if connum < len(con):
#             if con[connum] == cheknumb:
#                 sol.update({i:1})
#                 connum += 1
#             else:
#                 sol.update({i:0})
#         else:
#                 sol.update({i:0})
#         cheknumb += 1


In [6]:
def Sector_check(w_dic):

    Sector_sum_dic = {}
    d_sum_dic = {}
    d_sum_dic2 = {}
    #date=date_list[0]
    #using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP=preprocessing(date)

    for i in dic_sector:
#         updatenum = -0.1
        d_sum_dic.update({i:0})
        d_sum_dic2.update({i:0})
#         for j in dic_sector[i]:
#             updatenum += dic_bench[j]
#         Sector_sum_dic.update({i:updatenum})

    #print(Sector_sum_dic)

    for i in risk_sedol:
        if w_dic[i] > 0:
            savenum = d_sum_dic[dic_aseet_sector[i]]
            savenum += 0.05
            d_sum_dic.update({dic_aseet_sector[i]:savenum})
            savenum = 0
        else:
            savenum = d_sum_dic[dic_aseet_sector[i]]
            savenum = savenum - dic_bench[i]
            d_sum_dic.update({dic_aseet_sector[i]:savenum})
            savenum = 0
    
            
    in_Sector = -999
    out_Sector = -999
    for i in d_sum_dic:
        if d_sum_dic[i] < -0.01:
            in_Sector = i
            print(d_sum_dic[i])
            print(d_sum_dic)
            out_Sector = max(d_sum_dic.keys(), key=(lambda k: d_sum_dic[k]))
            print("in_Sector : ",in_Sector,"  out_Sector : ",out_Sector)
            return in_Sector,out_Sector
            break
    
    
        
        
        
    if in_Sector == -999:
#         for i in risk_sedol:
#             if w_dic[i] > 0:
#                 savenum = d_sum_dic2[dic_aseet_sector[i]]
#                 if 0.01 - dic_bench[i] > -0.05:
#                     savenum = savenum + 0.01 - dic_bench[i]
#                     d_sum_dic2.update({dic_aseet_sector[i]:savenum})
#                     savenum = 0
#                 else:
#                     savenum = savenum -0.05
#                     d_sum_dic2.update({dic_aseet_sector[i]:savenum})
#                     savenum = 0
#             else:
#                 savenum = d_sum_dic2[dic_aseet_sector[i]]
#                 savenum = savenum - dic_bench[i]
#                 d_sum_dic2.update({dic_aseet_sector[i]:savenum})
#                 savenum = 0
        
#         for i in d_sum_dic2:
#             if d_sum_dic2[i] > 0.01:
#                 out_Sector = i
#                 in_Sector = min(d_sum_dic2.keys(), key=(lambda k: abs(d_sum_dic2[k])))
#                 print(d_sum_dic2[i],"2222222222222222222222222")
#                 print("in_Sector : ",in_Sector,"  out_Sector : ",out_Sector)
#                 return in_Sector,out_Sector
#                 break
        print("nono se")
        return 100  
#     if in_Sector == -999:
        
    

def Sector_change2(sol,inS,outS):
    w_list = []
    
    for i in risk_sedol:
        w_list.append(sol[i])


    print("inS : ",inS)
    print("outS : ",outS)
    
    diff_sub = np.zeros((len(as_),len(as_)))

    candi = []
    indnum = 0
    indnum2 = 0
#     risk[A][A] + risk[A][E] +risk[E][A]
    for i in risk_sedol :
        if sol[i] > 0 and dic_aseet_sector[i] == outS:
            for j in risk_sedol :
                if sol[j] == 0 and dic_aseet_sector[j] == inS:
                    diff_sub[indnum][indnum2] = risk_mat[indnum][indnum] + 2*risk_mat[indnum][indnum2] - alpha[indnum] + alpha[indnum2]
                    
                indnum2 += 1
        indnum += 1
        indnum2 = 0


    print(np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0])
    print(np.where(diff_sub == min(diff_sub[diff_sub>0]))[1][0])

    w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[1][0]] = 0.1

    for val in range(len(w_list)) :
        if w_list[val] > 0. :
            candi.append(val)
    print(candi)
    #print("candi")

    return candi


def Sector_change(sol,inS,outS):
    w_list = []
    
    for i in risk_sedol:
        w_list.append(sol[i])


    print("inS : ",inS)
    print("outS : ",outS)
    
    diff_out = np.array([0]*len(as_), float)
    diff_sub = np.array([0]*len(as_), float)
    

    candi = []
    indnum = 0
    indnum2 = 0
    indnum3 = 0
    indnum4 = 0
#     risk[A][A] + risk[A][E] +risk[E][A]

    chek_one = 0

    for i in risk_sedol :
        if sol[i] > 0 and dic_aseet_sector[i] == outS:
            chek_one += 1
            save_risk = 0
            for j in risk_sedol :
                if sol[j] > 0 and dic_aseet_sector[j] == outS:
                    save_risk += risk_mat[indnum][indnum2]                                
                indnum2 += 1
            save_risk += alpha[indnum]

            diff_out[indnum] = save_risk 
        indnum += 1
        indnum2 = 0
    
    if chek_one == 1:
        in_cady = -99999999999999999999
    
    w_list[np.where(diff_out == max(diff_out[diff_out>0]))[0][0]] = 0   
    
    print(np.where(diff_out == max(diff_out[diff_out>0]))[0][0])
    
    in_cady = np.where(diff_out == max(diff_out[diff_out>0]))[0][0]
    
    for i in risk_sedol :
        if sol[i] == 0 and dic_aseet_sector[i] == inS:
            save_risk = 0            
            for j in risk_sedol :
                if sol[j] > 0 and dic_aseet_sector[j] == outS and indnum4 != in_cady:

                    save_risk += risk_mat[indnum3][indnum4]
                indnum4 += 1
            save_risk += alpha[indnum3]
            save_risk += risk_mat[indnum3][indnum3]
            diff_sub[indnum3] = save_risk                     
        indnum3 += 1
        indnum4 = 0


    
    print(np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0])


    w_list[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0.1

    for val in range(len(w_list)) :
        if w_list[val] > 0. :
            candi.append(val)
    print(candi)
    #print("candi")

    return candi

# date=date_list[0]
# using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector=preprocessing(date)

# as_ = using_dic['as']
# sol = {}
# indnummm = 0

# for i in risk_sedol:
#     if indnummm < 70:
#         sol.update({i:1})
#     else:
#         sol.update({i:0})
#     indnummm += 1
    

# for j in range(10):
#     con = Sector_change(sol)
#     cheknumb = 0
#     connum = 0
#     sol = {}
#     for i in risk_sedol:
#         if connum < len(con):
#             if con[connum] == cheknumb:
#                 sol.update({i:1})
#                 connum += 1
#             else:
#                 sol.update({i:0})
#         else:
#                 sol.update({i:0})
#         cheknumb += 1


solve!

In [13]:
#for i in date: 이런식으로 하되 evaluation criteria 는 아직 반영안함..!
feasi = 0
infeasi = 0
list_feasi_obj = []
n = 0
fini = True
k = 0
w_pre = {}
opt_return_list = []

############################################
while(fini):

    
    print(n, "period")

    date=date_list[n]
    init_set='GAN' #initial set by GAN, cplex, random

    c_time = 10. #cplex time for initial set
    gan_rot = 1800 #GAN epoch size for initial set
############################################


#preprocessing
    using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha,dic_aseet_MCAP,dic_aseet_sector,dic_risk_index=preprocessing(date)


#set initial cardinality
    if init_set=='cplex':
        Q_con, qmat, alpha = init_cplex(risk_mat,risk_sedol)
        sol = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,
                MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,multiple=10000,time_init=90)[0][0]
        alpha = []


        for i in risk_sedol:

            alpha.append(-1*dic_sedol_as[i])

    elif init_set=='GAN':

        bw=list(using_dic['bw'])
        bw=np.array(bw)

        beta=list(using_dic['beta'])
        beta=np.array(beta)

        alpha=list(using_dic['as'])
        alpha=np.array(alpha)

        sector_set = set(list(using_dic['sector']))
        sector_list = list(using_dic['sector'])

        gbs=using_dic[['sector', 'index']].apply(tuple, axis=1)
        group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

        mq_list=list(using_dic['mq'])
        mq_set=set(list(using_dic['mq']))

        gbm=using_dic[['mq', 'index']].apply(tuple, axis=1)
        group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]

        nbasset=len(bw)
        nbsector=len(sector_set)
        nbmq=len(mq_set)
        X_dim = nbasset
        HL_dim = 50
        Z_dim = 10

        np.random.seed()
    #     G = Generator(Z_dim, HL_dim, X_dim)

    #     sample_D = Discriminator(X_dim, train=False)
    #     train_D = Discriminator(X_dim, train=True)

        w_pre_gan=np.zeros(len(asset_list))

        for i,j in enumerate(asset_list):
            if j in w_pre.keys(): w_pre_gan[i]=w_pre[j]
            else:w_pre_gan[i]=0.

        car=50

        G = generator.Generator(Z_dim, HL_dim, X_dim)


        sample_D = discriminator.Discriminator(in_dim=X_dim, train=False, risk_mat=risk_mat, bw=bw, beta=beta, alpha=alpha, w_11=0.1
                                               ,w_pre_gan=w_pre_gan,MONTH=n,c_lower=car)
        train_D = discriminator.Discriminator(in_dim=X_dim, train=True, risk_mat=risk_mat, bw=bw, beta=beta, alpha=alpha, w_11=0.1
                                              ,w_pre_gan=w_pre_gan,MONTH=n,c_lower=car)
        G_solver = optim.Adam(G.parameters(), lr=2e-3)

        mb_size = 128

        best_obj = 100000.0
        best_sols = []

        losses = []

        start_time = time.time()

        feasible_threshold = 0.95

        adaptive_inf_scale = float(nbasset)
        rotation=1800

        for it in range(rotation):
            if it % 900 == 0:
                sample_noise = generator.sample_Z(128, Z_dim)
                samples = G(sample_noise)

                sample_D.Zstar = best_obj
                sample_D.inf_scale = adaptive_inf_scale

                sample_D_values = sample_D(samples).data.numpy()
                sample_obj_values = sample_D.objs.data.numpy()

                fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
                sols = samples.data.numpy()[sample_D_values > feasible_threshold]

                if len(fea_sample_obj_values > 0):
                    best_idx = np.argmin(fea_sample_obj_values)
                    if np.min(fea_sample_obj_values) < best_obj:
                        best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                        best_sols.append(sols[best_idx])

            z = Variable(torch.randn(mb_size, Z_dim))

            train_D.Zstar = best_obj
            train_D.inf_scale = adaptive_inf_scale

            G_loss = -torch.mean(torch.log(train_D(G(z))))

            G.zero_grad()
            G_loss.backward()
            G_solver.step()

        best,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)
        sol={}
        for i in range(len(risk_sedol)):
            if i in best: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.



    elif init_set=='random':
        np.random.seed()
        r=list(set(list(np.random.randint(len(risk_sedol),size=70))))
        sol={}
        for i in range(len(risk_sedol)):
            if i in r: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.


# solve & update
    sol=sol
    as_ = using_dic['as']
#     opt_return = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,
#                 MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,multiple=10000,time_init=90,cad_set = sol)[0][2]
#     opt_return_list.append(opt_return)
    if n == 0:
        w_pre = {}
        for j in risk_sedol:
            w_pre.update({j:0})
        results = update(sol,80.,as_,w_pre,0)
            
    else:
        results = update(sol,80.,as_,w_pre,1)

    result = results[0]
    
    try:
        w_index = result.index(min(result))
        result2 = results[1][w_index]
        print(result2)

        w_pre = w_i_pre_sum(result2)
        feasi += 1
        list_feasi_obj.append(result)
        print("------objective value 변화------")
        print(result)
        n += 1
        
    except:
        infeasi += 1
        
    
        
    if n == 1:
        fini = False
    k += 1
print("feasible 횟수 : ",feasi)
print("저장된 objective value")
print(list_feasi_obj)
print("infeasible 횟수 : ",infeasi)
print(opt_return_list)

0 period
0 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
[13, 36, 40, 45, 57, 63, 86, 88, 116, 127, 130, 180, 182, 207, 212, 214, 223, 233, 249, 263, 268, 272, 273, 276, 282, 286, 296, 300, 307, 316, 317, 323, 344, 346, 349, 351, 355, 380, 382, 402, 405, 417, 438, 441, 456, 457, 459, 468, 480, 484]
50
con
cplex no solution
-0.070986429
{'Consumer Discretionary': 0.44373453400000001, 'Health Care': -0.07098642899999999, 'Utilities': 0.17136111199999998, 'Information Technology': 0.30481401099999994, 'Materials': 0.17498660399999999, 'Financials': 0.13532836600000001, 'Industrials': 0.20198778199999995, 'Energy': 0.067877613000000003, 'Consumer Staples': 0.16593949399999994, 'Telecommunication Services': -0.023970735999999999}
in_Sector :  Health Care   out_Sector :  Consumer Discretionary
inS :  Health Care
outS :  Consumer Discretionary
484
360
[13, 36, 40, 45, 57, 63, 86, 88, 116, 127, 130, 180, 182, 207, 212, 214, 223, 233, 249, 263, 268, 272, 273, 276, 282, 286, 296, 300, 307,

O : 24600000.0
19.659844538231855
AC
0.900028412
TE
[ 19.70148997]
0.32267603346  wupsum
1.00000119662  wsum!
475  wupdic_len
[ 0.1970149]
0.802985100295
O : 24600000.0
26.696272383552717
TE :  [ 26.74614428]
AS :  0.900028412
Bs infeasible
O : 24600000.0
22.906692207783177
TE :  [ 22.9513161]
AS :  0.900028412
O : 24600000.0
24.292120072899877
TE :  [ 24.33935988]
AS :  0.900028412
O : 24600000.0
24.739003048763337
TE :  [ 24.78660497]
AS :  0.900028412
O : 24600000.0
25.469127573878957
TE :  [ 25.51699476]
AS :  0.900028412
Bs infeasible
O : 24600000.0
24.900823646365644
TE :  [ 24.94857692]
AS :  0.900028412
infeasible!
TE infeasible
TE_out :  212
cplex no solution
nono se
nono MC
346
462
[45, 57, 60, 63, 86, 88, 102, 116, 127, 180, 182, 207, 212, 233, 249, 261, 263, 272, 273, 276, 282, 286, 296, 300, 304, 316, 317, 323, 336, 344, 349, 351, 360, 363, 369, 380, 382, 402, 405, 411, 417, 431, 438, 441, 456, 459, 462, 468, 472, 480]
O : 24600000.0
20.19754290625811
AC
0.900577344
TE
[ 2

O : 24600000.0
25.14073766356739
TE :  [ 25.18932616]
AS :  0.893879951624
O : 24600000.0
24.486286272271638
TE :  [ 24.53572075]
AS :  0.893880528913
BS Feasible!!!
obj_val
25.2379146513
229
212
[5, 8, 11, 12, 15, 16, 20, 31, 47, 48, 55, 70, 73, 81, 90, 109, 115, 129, 137, 139, 145, 149, 155, 158, 159, 168, 171, 177, 179, 192, 197, 211, 212, 228, 253, 265, 277, 293, 308, 340, 348, 363, 369, 396, 411, 422, 437, 461, 462, 463]
cplex_val
25.14073766356739
O : 24600000.0
23.348446669800527
AC
0.893446801391
TE
[ 23.40030012]
0.247592879355  wupsum
1.00000368112  wsum!
474  wupdic_len
[ 0.234003]
0.765996998781
O : 24600000.0
66.23981047956481
TE :  [ 66.28200025]
AS :  0.893232945
Bs infeasible
O : 24600000.0
26.12636954925965
TE :  [ 26.17746329]
AS :  0.893938906207
O : 24600000.0
23.993477057665018
TE :  [ 24.04553963]
AS :  0.893446802208
O : 24600000.0
24.806131899309463
TE :  [ 24.85872858]
AS :  0.893701227179
O : 24600000.0
25.38384749792393
TE :  [ 25.43573968]
AS :  0.8939389597

In [14]:
chchlist = []
sumsum = 0
cadcad = 0
for i in range(len(list_feasi_obj)):
    for j in range(len(list_feasi_obj[i])):
        chchlist.append(list_feasi_obj[i][j])
        sumsum += list_feasi_obj[i][j]
        cadcad += 1

print(min(chchlist))
print(max(chchlist))
print(sumsum/cadcad)

24.999601253230647
42.239238256905836
26.188110514068207


Evaluation Criteria

In [7]:
def w_i_pre_sum(w_dic):
   
    w_pre = 0  
    
    for i in w_dic.keys():


        w_pre += w_dic[i]*(dic_r[i]+1)
     
        
    w_i_pre_dic = {}

    for i in w_dic.keys():
        w_i_pre_dic.update({i:(float(w_dic[i])*(dic_r[i]+1))/w_pre})
#         print((float(w_dic[i])*(dic_sedol_as[i]+1))/w_pre)

    
    return w_i_pre_dic




def turnover(w_dic,w_dic2):
    pre_list = set(list(w_dic.keys()))

    rotp_t0 = 0
    
    for i in risk_sedol:
        rotp_t0 += w_dic[i]*dic_r[i]
        
    w_i_pre = {}
    
    w_pre = 0
    
    for i in risk_sedol:
        w_pre += w_dic[i]*(dic_r[i]+1)

        
        
    w_i_pre_sum = 0
    
    for i in risk_sedol:
        w_i_pre.update({i:(w_dic[i]*(dic_r[i]+1))/w_pre})
        w_i_pre_sum += (w_dic[i]*(dic_r[i]+1))/w_pre

        

   
        
    
    pre_list2 = set(list(w_dic2.keys()))

    turnover = 0

    add_key_1 = pre_list-pre_list2
    add_key_2 = pre_list2-pre_list


    for i in add_key_2:
        w_dic.update({i:0})

    for i in add_key_1:
        w_dic2.update({i:0})

    pre_list.union(pre_list2)

    for i in pre_list:
        turnover += abs(w_dic2[i]-w_i_pre[i])

    rOTP = 0

    for i in dic_sedol_as.keys():
        rOTP += w_dic2[i]*dic_sedol_as2[i]

    r_Txadj_t = rOTP - turnover*0.5
    
    result=[]
    result.append(turnover,rOTP,r_Txadj_t)
    result.append(w_i_pre_sum)
    
    print("turnover : " + str(turnover))
    print("r_OTP : " + str(rOTP))
    print("r_Txadj_t : " + str(r_Txadj_t))
    
    return result


def IR_TE(result,bench_t):
    cal1 = 1
    cal2 = 1
    list_std=np.array([])

    for i in range(len(result)):
        cal1 = cal1*(1+result[i])
        cal2 = cal2*(1+bench_t[i])
        list_std = np.append(list_std,resultp[i]-bench_t[i])
    
    IR = (cal1-cal2)/np.std(list_std)
    
    TE = math.sqrt(13)*np.std(list_std)
    
    result = []
    
    result.append(IR,TE)
    
    return result

def R_SR(result,bench,n):
    R_CUM = 1
    B_CUM = 1
    for i in range(len(result)):
        R_CUM = R_CUM*(1+result[t])
        
    for i in range(len(bench)):
        B_CUM = B_CUM*(1+bench[t])
    
    R_CUM = R_CUM-1
    
    B_CUM = B_CUM-1
    
    R_ANN = (1+R_CUM)**(13/n) - 1
    
    B_ANN = (1+B_CUM)**(13/n) - 1
    
    R_EXAN = R_ANN-B_ANN
    
    r_t = np.array(result)
    
    SR = R_CUM/np.std(r_t)
    
    

함수정의 ipp 적용

In [45]:
def cal_obj(sol, risk_mat, as_) :
    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 10000)
    risk_mat = risk_mat * a
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, as_*10000)
    obj = s_r-s_a
    return obj


@ipp.require(cal_obj)
def select_candi(sol) :
    
    diff_sub = np.array([0]*len(as_), float)
    diff_add = np.array([0]*len(as_), float)
    candi = []

    for i in range(len(sol)) :
        if(sol[i] > 0) :
            w = sol[i]
            sol[i] = 0
            diff_sub[i] = cal_obj(sol, risk_mat, as_)
            return diff_sub[i]
            if min(diff_sub[diff_sub>0]) == diff_sub[i] :
                for j in range(len(sol)) :
                    if sol[j] == 0 :
                        sol[j] = w
                        diff_add[j] = cal_obj(sol, risk_mat, as_)
                        sol[j] = 0
            sol[i] = w
        
    sol[np.where(diff_sub == min(diff_sub[diff_sub>0]))[0][0]] = 0
    sol[np.where(diff_add == min(diff_add[diff_add>0]))[0][0]] = 0.1

    for val in range(len(sol)) :
        if sol[val] > 0. :
            candi.append(val)

    return candi


@ipp.require(select_candi)
def update(sol) :
    start_time = time.time()
    new_list = []
    con = []
    nums = 0
    for k in risk_sedol :
        new_list.append(sol[k])
        if sol[k] == 1:
            con.append(nums)
        nums += 1
    result_list = []
    result_list2 = []
    
    while(True):
        if time.time() - start_time > lim_time:
            break
        
        ex.set_con(cons=con)
        new_list2 = []
        global new_list3
        new_list3 = []
        try:
            aabbaa = ex.solve(0.01)
            if aabbaa != 0:
                aaaa = aabbaa[0]
                bbbb= aabbaa[1]
                for k in risk_sedol :
                    new_list2.append(aaaa[k])
                for k in risk_sedol :
                    new_list3.append(bbbb[k])
                new_list = new_list2
                result_list.append(aabbaa[2])
                result_list2.append(aabbaa[0])
                con = select_candi(new_list)
            else:
                print("TE")
#                 con = select_candi(new_list)
        except:
            print("NO")
#             con = select_candi(new_list)
    return result_list,result_list2


def sample_Z(m, n):
    return Variable(torch.Tensor(np.random.uniform(0., 1., size=[m, n])))

        
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return np.random.normal(size=size, scale=xavier_stddev)


class Generator(nn.Module):
    def __init__(self,  in_dim=10, hd_dim=50, out_dim=500):
        super(Generator, self).__init__()
        
        self.G_W1 = nn.Parameter(torch.from_numpy(xavier_init([in_dim, hd_dim])).float())
        self.G_b1 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
    
        self.G_W12 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, hd_dim])).float())
        self.G_b12 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
        self.G_W13 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, hd_dim])).float())
        self.G_b13 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())

        
        self.G_W2 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, out_dim])).float())
        self.G_b2 = nn.Parameter(torch.from_numpy(np.zeros(shape=[out_dim])).float())

    def forward(self, x):
        x = F.relu( x @ self.G_W1 + self.G_b1)
        x = F.relu( x @ self.G_W12 + self.G_b12)
        x = F.relu( x @ self.G_W13 + self.G_b13)
        x = x @ self.G_W2 + self.G_b2

        return F.sigmoid(x*0.1)


class Discriminator(nn.Module):
    def __init__(self, in_dim=500,train=False):
        super(Discriminator, self).__init__()
        self.tot_infs = 0.0
        self.tot_infs_nor = 0.0
        self.train = train
        self.x_threshold = 0.001
        self.inf_scale = 100.0
        self.Zstar = 1000
        
        self.in_dim = in_dim

        np.random.seed(0)
        self.Omega = Variable(torch.from_numpy(risk_mat)).float()

        self.card_upper = 70.
        self.card_lower = 69.
        
        
    def forward(self, x):
        tot_infs = Variable(torch.zeros(x.size()[0]))
        x_bw = Variable(torch.from_numpy(bw).float())
        alp = Variable(torch.Tensor(alpha).float())   
        bet = Variable(torch.Tensor(beta).float())
 
        # (4)
        tot_infs += F.relu(1. - torch.sum(x,1))
        tot_infs += F.relu(torch.sum(x,1) - 1.)
        
        # (5)
        tot_infs += F.relu(torch.abs(x-x_bw)@ Variable(torch.ones(self.in_dim))-0.05)

        # (6)
        for j in range(nbsector):
            k= (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_sector[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (7)
        for j in range(nbmq):
            k = (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_mq[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (8)
        k = (x - x_bw)@bet
        l = torch.abs(k)
        tot_infs += F.relu(l - 0.1)
        
        # (9) lb <= card(x) <= b
        if self.train:
            num_non_zeros = torch.sum(torch.tanh(x / self.x_threshold), 1)
            tot_infs += F.relu(num_non_zeros - self.card_upper) + F.relu(self.card_lower - num_non_zeros)
        else:
            num_non_zeros = torch.sum(x.data>self.x_threshold, 1).float()
            tot_infs += Variable(np.maximum(num_non_zeros - self.card_upper, 0) + np.maximum(self.card_lower - num_non_zeros, 0))
    
        
        # (10)
        l = 0.5*torch.sum(torch.abs(x - x_bw))
        tot_infs += F.relu(0.6 - l)
        #tot_infs += F.relu(l - 1.)
    
        
        # (11) a<=dQd <= b
        dQ = (x-x_bw) @ self.Omega
        size_x = x.size()
        dQd = torch.bmm(dQ.view(size_x[0],1,size_x[1]), (x-x_bw).view(size_x[0],size_x[1],1)).squeeze(1).squeeze(1)
        tot_infs += F.relu(dQd - 0.01)
        tot_infs += F.relu(0.0025 - dQd)
        
        
        # min dQd + ad (<= Zstar)
        l = (x-x_bw)@alp

        MONTH =0
        turnover=x_bw
        
        if MONTH > 0 :
            tot_infs += (F.relu(100.*dQd - 100.*l -0.5*turnover - self.Zstar))
            self.objs = 100.*dQd - 100.*l -0.5*turnover
        else:
            tot_infs += (F.relu(50.*dQd - 50.*l  - 0.5*self.Zstar ))
            self.objs = 100.*dQd - 100.*l
        
        
        fea_probs = F.relu(1. - F.tanh(tot_infs/self.inf_scale))  

        
        return fea_probs
        


def GAN(rotation):
    X_dim = nbasset
    HL_dim = 50
    Z_dim = 10

    G = Generator(Z_dim, HL_dim, X_dim)

    sample_D = Discriminator(X_dim, train=False)
    train_D = Discriminator(X_dim, train=True)

    G_solver = optim.Adam(G.parameters(), lr=2e-3)

    mb_size = 128

    best_obj = 100000.0
    best_sols = []

    losses = []

    start_time = time.time()

    feasible_threshold = 0.95

    adaptive_inf_scale = float(nbasset)

    for it in range(rotation):
        if it % 900 == 0:
            sample_noise = sample_Z(128, Z_dim)
            samples = G(sample_noise)

            sample_D.Zstar = best_obj
            sample_D.inf_scale = adaptive_inf_scale

            sample_D_values = sample_D(samples).data.numpy()
            sample_obj_values = sample_D.objs.data.numpy()

            fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
            sols = samples.data.numpy()[sample_D_values > feasible_threshold]

            if len(fea_sample_obj_values > 0):
                best_idx = np.argmin(fea_sample_obj_values)
                if np.min(fea_sample_obj_values) < best_obj:
                    best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                    best_sols.append(sols[best_idx])

        z = Variable(torch.randn(mb_size, Z_dim))

        train_D.Zstar = best_obj
        train_D.inf_scale = adaptive_inf_scale

        G_loss = -torch.mean(torch.log(train_D(G(z))))

        G.zero_grad()
        G_loss.backward()
        G_solver.step()
        
    sam=[]
    for i in range(50):
        n = sample_Z(128, 10)
        samples = G(n)
        sam.append(samples)
        
    return best_sols, sols,fea_sample_obj_values,sam

        

def GAN_reulst(best_sols, sols,fea_sample_obj_values):    
    good_sols=np.argsort(fea_sample_obj_values)[:10]
    card_list=[list(np.where(sols[i]>0.001)[0]) for i in range(len(good_sols))]
    card_freq=np.zeros(nbasset)
    for i in range(len(card_list)):
        for j in card_list[i]:
            card_freq[j]+=1
    best=np.argsort(sum(sols))[::-1][:len(card_freq[card_freq>len(card_list)-5])]
    bestsol=np.where(best_sols[0]>0.001)[0]
    return best,bestsol

def init_cplex(risk_mat,risk_sedol):
    sedol_var_list =[]
    for i in risk_sedol:
        sedol_var_list.append("d"+str(i))

    for i in risk_sedol:
        sedol_var_list.append("q"+str(i))

    sedol_var_list.append("assum")
    #global alpha
    alpha = []
    for i in risk_sedol:
        alpha.append(-10000.*dic_sedol_as[i])

    qmat=[]
    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(20000*j)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(len(risk_mat)):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    for i in range(1):
        qmat_1=[]
        qmat_1.append(sedol_var_list)
        new_risk_mat=[]
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        for j in risk_mat[i]:
            new_risk_mat.append(0)
        new_risk_mat.append(0)
        qmat_1.append(new_risk_mat)
        qmat.append(qmat_1)

    q_con1 = []
    q_con2 = []
    q_val = []


    for i in range(len((risk_mat[0]))):
        for j in range(len(risk_mat[0])):
            if j >= i:
                q_con1.append(i)
                q_con2.append(j)
                if i == j:
                    ex_list = list(risk_mat[i])
                    q_val.append(0.5*ex_list[j]*10000)
                else:
                    ex_list = list(risk_mat[i])
                    q_val.append(ex_list[j]*10000)

    Q_con = []
    Q_con.append(q_con1)
    Q_con.append(q_con2)
    Q_con.append(q_val)
    
    return Q_con, qmat, alpha

print("complete")

complete


solve 적용

In [22]:
#for i in date: 이런식으로 하되 evaluation criteria 는 아직 반영안함..!
feasi = 0
infeasi = 0
list_feasi_obj = []
list_feasi_sol = []

############################################
n=0
date=date_list[n]
init_set='ipp' #initial set by GAN, cplex, random

c_time = 10. #cplex time for initial set
gan_rot = 1800 #GAN epoch size for initial set
############################################

using_dic,asset_list,dic_sedol_as,dic_bench,dic_beta,dic_sector,dic_MCAP,risk_sedol,risk_mat,dic_r,alpha=preprocessing(date)
as_ = using_dic['as']
    
dv['as_'] = as_
dv['w_pre'] = w_pre
dv['dic_sector'] = dic_sector
dv['dic_bench'] = dic_bench
dv['risk_sedol'] = risk_sedol
dv['dic_MCAP'] = dic_MCAP
dv['dic_beta'] = dic_beta
dv['alpha'] = alpha
dv['risk_mat'] = risk_mat
dv['using_dic'] = risk_mat

# solve & update
for k in range(1):
    print("solve print : ", k, "번째")
    #set initial cardinality
    if init_set=='cplex':
        Q_con, qmat, alpha = init_cplex(risk_mat,risk_sedol)
        sol = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,
                MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000,time_init=90)[3][0]
        alpha = []


        for i in risk_sedol:

            alpha.append(-1*dic_sedol_as[i])
    elif init_set=='GAN':

        bw=list(using_dic['bw'])
        bw=np.array(bw)

        beta=list(using_dic['beta'])
        beta=np.array(beta)

        alpha=list(using_dic['as'])
        alpha=np.array(alpha)

        sector_set = set(list(using_dic['sector']))
        sector_list = list(using_dic['sector'])

        gbs=using_dic[['sector', 'index']].apply(tuple, axis=1)
        group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

        mq_list=list(using_dic['mq'])
        mq_set=set(list(using_dic['mq']))

        gbm=using_dic[['mq', 'index']].apply(tuple, axis=1)
        group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]

        nbasset=len(bw)
        nbsector=len(sector_set)
        nbmq=len(mq_set)

        best_sols, sols,fea_sample_obj_values,samples=GAN(gan_rot)
       # best,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)

        #print(len(best))


        aa=np.zeros(nbasset)
        for i in range(50):
            sb =samples[i].data.numpy()
            bb=sum(sb>0.0001)
            aa=aa+bb
        general=np.where(aa.max()*0.001<aa)
        good=np.where(aa.max()*0.1<aa)
        plus=list(set(general[0])-set(good[0]))



        best=list(good[0])
        sol={}
        for i in range(len(risk_sedol)):
            if i in best: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.


#         best1=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))
#         sol1={}
#         for i in range(len(risk_sedol)):
#             if i in best1: sol1[risk_sedol[i]]=1.
#             else: sol1[risk_sedol[i]]=0.

#         best2=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
#         sol2={}
#         for i in range(len(risk_sedol)):
#             if i in best2: sol2[risk_sedol[i]]=1.
#             else: sol2[risk_sedol[i]]=0.

#         best3=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
#         sol3={}
#         for i in range(len(risk_sedol)):
#             if i in best3: sol3[risk_sedol[i]]=1.
#             else: sol3[risk_sedol[i]]=0.

#         best4=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
#         sol4={}
#         for i in range(len(risk_sedol)):
#             if i in best4: sol4[risk_sedol[i]]=1.
#             else: sol4[risk_sedol[i]]=0.

#         best5=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))        
#         sol5={}
#         for i in range(len(risk_sedol)):
#             if i in best5: sol5[risk_sedol[i]]=1.
#             else: sol5[risk_sedol[i]]=0.       

#         best6=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))                
#         sol6={}
#         for i in range(len(risk_sedol)):
#             if i in best6: sol6[risk_sedol[i]]=1.
#             else: sol6[risk_sedol[i]]=0.       

#         best7=list(good[0])+list(np.random.choice(plus, np.random.randint(3,5)))                
#         sol7={}
#         for i in range(len(risk_sedol)):
#             if i in best7: sol7[risk_sedol[i]]=1.
#             else: sol7[risk_sedol[i]]=0.

    elif init_set=='random':
        r=list(set(list(np.random.randint(len(risk_sedol),size=70))))
        sol={}
        for i in range(len(risk_sedol)):
            if i in r: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.
    
    elif init_set=='ipp':
        sol = {'2295677': 0.0, '2778844': 0.0, '8066051': 1.0, '2342034': 0.0, '2417868': 1.0, '2034524': 0.0, 'B1Z77F6': 0.0, 'B28V183': 0.0, '2914734': 0.0, '2820563': 1.0, '2795393': 0.0, 'B3KFWW1': 1.0, 'B3Q2FJ4': 0.0, 'B40K911': 0.0, 'B4NH4W5': 0.0, 'B5BKPQ4': 1.0, 'B5BT0K0': 0.0, 'B633030': 0.0, 'B63QTN2': 0.0, 'B6TFK30': 0.0, '2779397': 0.0, 'B6WVMH3': 1.0, 'B76FBQ0': 0.0, 'B7QQYV9': 0.0, 'B7VD3F2': 0.0, 'B840261': 0.0, 'B84LSN3': 0.0, 'B8DMK08': 0.0, 'B8KQN82': 0.0, 'B92RRW2': 0.0, '2018175': 0.0, '2422442': 0.0, 'B9M2WX3': 0.0, 'BBM4S68': 1.0, '2417987': 0.0, '2162845': 0.0, '2963899': 0.0, 'BD5JS97': 0.0, 'BD95VZ8': 0.0, 'BDHLTQ5': 1.0, '2824770': 0.0, 'BF0FM92': 0.0, 'BF0HZC2': 0.0, 'BF6V3T1': 0.0, 'BFG3KF2': 1.0, '2320524': 0.0, '2575896': 0.0, '2983604': 0.0, '2605555': 0.0, 'BJWG213': 0.0, '2767165': 0.0, '2101503': 0.0, 'BLNN369': 0.0, 'BMH4NJ8': 0.0, '2819118': 0.0, '2692632': 0.0, 'BSPHGL4': 1.0, '2331225': 0.0, 'BTN1Y11': 0.0, 'BTN1Y44': 0.0, '2522096': 1.0, 'BVFMFG8': 0.0, 'BWSW5J5': 0.0, '2367480': 1.0, 'BY7QL61': 0.0, 'BY9D546': 0.0, 'BYND5N1': 0.0, 'BYND5W0': 0.0, 'BYNF418': 1.0, 'BYP66M4': 0.0, 'BYQQ3P5': 0.0, 'BYV2325': 0.0, 'BYVY8G0': 0.0, 'BYWTW73': 1.0, 'BYX4D52': 0.0, 'BYXB201': 0.0, 'BYXD7B3': 0.0, 'BYZGYC0': 1.0, 'BZ0P3Z5': 0.0, 'BZ2Y3W7': 0.0, 'BZBY209': 0.0, '2520153': 0.0, '2769796': 0.0, '2341484': 0.0, '2624486': 0.0, '2252058': 0.0, '2517382': 1.0, '2714923': 0.0, '2880255': 0.0, '2023737': 0.0, '2210614': 1.0, '2017213': 0.0, '2218447': 0.0, '2154927': 0.0, '2884224': 0.0, '2511908': 0.0, '2319157': 0.0, '2816409': 0.0, '2654320': 0.0, '2229676': 0.0, '2429090': 0.0, '2588184': 0.0, '2648806': 0.0, '2098876': 0.0, '2903550': 0.0, '2216850': 0.0, '2516839': 0.0, '2246288': 0.0, '2599142': 0.0, '2367026': 0.0, '2933632': 1.0, '2965839': 0.0, '2477152': 0.0, '2026082': 0.0, '2818740': 0.0, '2365161': 0.0, '2681511': 0.0, 'B198391': 0.0, '2588173': 0.0, '2790611': 0.0, '2008154': 0.0, '2047317': 0.0, '2185046': 0.0, '2804211': 0.0, '2885937': 0.0, '2215460': 0.0, '2235822': 1.0, '2550161': 0.0, '2936921': 0.0, '2041364': 1.0, '2466321': 0.0, '2707677': 0.0, '2434209': 0.0, '2885700': 0.0, '2550707': 0.0, '2476193': 0.0, '2195722': 0.0, '2632650': 0.0, '2236911': 1.0, '2880501': 0.0, '2261526': 0.0, '2803014': 1.0, '2480138': 1.0, '2365804': 0.0, '2871505': 1.0, '2073408': 0.0, '2066721': 0.0, '2034494': 0.0, '2295172': 0.0, '2649100': 0.0, '2261203': 0.0, '2758051': 0.0, '2113434': 0.0, '2110703': 0.0, '2769503': 0.0, '2578516': 0.0, '2967181': 0.0, '2038430': 0.0, '2639509': 0.0, '2280220': 0.0, '2681403': 1.0, '2570200': 0.0, '2077905': 1.0, '2717320': 0.0, '2511920': 0.0, '2767381': 0.0, '2465254': 0.0, '2232793': 1.0, '2909730': 0.0, '2346508': 0.0, '2842255': 0.0, '2665861': 0.0, '2829601': 0.0, '2126335': 0.0, '2183244': 0.0, '2767228': 1.0, '2039831': 0.0, '2284761': 0.0, 'B1FLZ21': 0.0, '2407966': 1.0, '2670519': 0.0, '2807566': 0.0, '2692397': 1.0, '2142784': 1.0, '2085878': 0.0, '2232696': 0.0, '2142922': 0.0, '2485070': 0.0, '2326618': 0.0, '2705024': 1.0, '2516378': 0.0, '2661568': 0.0, '2019479': 0.0, '2336747': 0.0, 'B067BM3': 0.0, '2672689': 0.0, '2378200': 0.0, '2101967': 0.0, '2182553': 1.0, '2883395': 0.0, 'B1F76F9': 1.0, '2684703': 0.0, '2577609': 0.0, '2214832': 0.0, '2634805': 0.0, 'B198380': 0.0, 'B01R311': 1.0, '2614807': 0.0, '2413758': 0.0, '2942188': 0.0, '2272205': 0.0, '2692665': 1.0, '2445966': 0.0, '2635701': 0.0, '2295945': 0.0, '2198163': 0.0, '2915500': 0.0, '2496113': 1.0, '2270726': 1.0, '2457552': 0.0, '2928683': 0.0, '2705648': 1.0, '2344331': 0.0, '2985677': 1.0, '2345022': 0.0, '2580986': 0.0, '2094670': 0.0, '2259468': 1.0, '2422806': 0.0, '2626921': 0.0, '2931205': 0.0, '2698470': 0.0, '2668804': 0.0, '2671501': 0.0, '2428008': 0.0, '2299011': 0.0, '2510723': 1.0, '2256511': 0.0, '2842040': 1.0, '2313405': 0.0, '2867913': 0.0, '2068921': 0.0, '2695921': 0.0, '2279303': 0.0, '2304227': 0.0, '2289874': 0.0, '2685717': 0.0, '2202536': 0.0, '2861078': 0.0, '2960384': 0.0, '2043694': 1.0, '2484088': 0.0, '2654461': 0.0, '2542049': 0.0, '2615468': 0.0, '2459020': 0.0, '2020459': 0.0, '2378534': 1.0, '2937689': 0.0, '2209106': 0.0, '2548616': 0.0, '2886907': 0.0, '2085102': 0.0, '2859954': 0.0, '2405302': 0.0, '2464972': 0.0, '2892090': 0.0, '2367598': 0.0, '2624356': 0.0, '2497406': 1.0, '2044545': 0.0, '2057059': 0.0, '2838555': 0.0, '2471789': 0.0, '2005973': 0.0, '2090173': 1.0, '2696838': 1.0, '2954019': 0.0, '2149309': 0.0, '2204026': 1.0, '2352118': 0.0, '2098597': 0.0, '2852533': 0.0, '2760669': 1.0, '2655408': 0.0, '2822600': 0.0, '2831543': 0.0, '2896713': 0.0, '2646015': 0.0, '2685007': 0.0, '2645409': 0.0, '2829515': 0.0, '2190385': 0.0, 'B011WL6': 0.0, '2536763': 0.0, '2220884': 0.0, '2699291': 0.0, '2192295': 0.0, '2680905': 0.0, '2910970': 0.0, '2160753': 0.0, '2333889': 0.0, '2305844': 0.0, '2765802': 0.0, '2589198': 0.0, '2250870': 0.0, '2319146': 0.0, '2941925': 1.0, '2814641': 0.0, '2445063': 0.0, '2269768': 0.0, '2958936': 0.0, '2069065': 0.0, '2884183': 0.0, '2573209': 0.0, '2853688': 0.0, '2922544': 0.0, '2032067': 0.0, '2732635': 0.0, '2380498': 0.0, '2636607': 0.0, 'B1L60G9': 0.0, '2377809': 0.0, '2011602': 1.0, '2310194': 0.0, '2249629': 0.0, '2650748': 0.0, '2108601': 0.0, '2510682': 0.0, '2086138': 0.0, '2766805': 0.0, '2073022': 0.0, '2026361': 0.0, '2463247': 0.0, '2611206': 0.0, '2680303': 0.0, '2625047': 0.0, '2032380': 1.0, '2023748': 0.0, '2937667': 0.0, '2831811': 0.0, '2440637': 0.0, '2002479': 0.0, '2002305': 0.0, '2963372': 0.0, '2105505': 0.0, '2630643': 0.0, '2491594': 0.0, '2641827': 0.0, '2350684': 0.0, '2050832': 0.0, '2019952': 0.0, '2028572': 0.0, '2523044': 0.0, 'B0SRLH6': 0.0, '2379504': 0.0, '2547044': 0.0, '2594437': 0.0, '2087807': 0.0, '2021690': 0.0, '2702791': 0.0, '2736035': 0.0, '2046552': 0.0, '2570545': 0.0, '2081779': 0.0, '2224701': 0.0, '2170473': 0.0, '2681801': 0.0, '2024020': 0.0, '2595708': 1.0, '2917766': 0.0, '2182779': 0.0, '2046251': 0.0, '2754060': 0.0, '2090571': 0.0, '2048804': 0.0, '2257019': 0.0, '2000019': 0.0, '2369174': 0.0, '2786159': 0.0, '2572303': 0.0, '2812452': 0.0, '2567741': 0.0, '2704407': 1.0, '2702337': 0.0, '2262314': 1.0, '2065308': 0.0, '2717483': 0.0, '2455965': 0.0, '2180201': 0.0, '2293819': 0.0, '2027342': 0.0, '2212870': 0.0, '2138158': 0.0, '2380863': 0.0, '2885409': 0.0, '2632003': 0.0, '2026242': 0.0, '2490911': 0.0, '2779201': 0.0, '2206657': 1.0, '2007849': 0.0, '2360326': 1.0, '2175672': 0.0, '2109002': 0.0, '2084637': 0.0, '2576350': 0.0, '2029809': 0.0, '2881537': 0.0, '2297907': 1.0, '2947956': 0.0, '2475833': 0.0, '2023607': 0.0, '2783815': 0.0, '2498993': 0.0, '2526117': 0.0, 'B0F0F09': 0.0, '2510615': 0.0, '2240202': 0.0, '2197137': 0.0, '2486813': 0.0, '2411138': 0.0, '2219224': 0.0, 'B0J7D57': 0.0, '2411053': 0.0, '2892045': 0.0, '2419240': 0.0, '2066408': 0.0, 'B08G9Z7': 0.0, '2480677': 0.0, '2641894': 1.0, '2868165': 0.0, '2312736': 0.0, '2100920': 0.0, '2065955': 0.0, '2651086': 0.0, '2511328': 0.0, 'B142VX8': 0.0, '2689560': 0.0, '2340168': 0.0, '2404569': 0.0, '2931595': 0.0, '2043962': 0.0, '2650793': 0.0, '2708841': 0.0, '2328915': 0.0, '2300601': 0.0, 'B00PWV9': 0.0, '2989301': 0.0, '2250687': 0.0, '2746829': 1.0, '2278407': 0.0, '2523022': 0.0, '2183330': 0.0, '2516152': 0.0, '2476803': 0.0, '2464165': 0.0, '2431846': 0.0, '2318024': 0.0, '2414580': 1.0, '2033004': 0.0, '2433842': 0.0, '2196888': 1.0, '2640147': 0.0, '2298386': 0.0, '2279002': 0.0, '2317087': 0.0, '2065159': 0.0, '2259101': 0.0, '2604206': 0.0, '2807492': 0.0, '2701271': 0.0, '2491839': 1.0, '2146838': 0.0, '2341860': 0.0, '2860990': 0.0, 'B12GHV2': 0.0, 'B0SRLF4': 0.0, '2648668': 0.0, '2841489': 1.0, '2655981': 0.0, '2334150': 0.0, '2690506': 0.0, '2182348': 1.0, '2199326': 0.0, '2196479': 0.0, '2859868': 0.0, '2989828': 0.0, '2586122': 0.0, '2674458': 0.0, '2830904': 0.0, '2027104': 0.0}
        sol_list = [sol]*8
        dv.execute('lim_time = 80.')
        
    if n == 0:
        w_pre = {}
        for j in risk_sedol:
            w_pre.update({j:0})
        
        dv.execute('O_scale = 0')
        dv.execute('ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)')
        dv.execute(' ex.set_omega(risk_mat)')
        results = dv.map(update, sol_list)
            
    else:
        dv.execute('O_scale = 1')
        dv.execute('ex = inform.informs(10000, 10000, w_pre, O_scale,dic_sector,dic_bench,risk_sedol,dic_MCAP,dic_beta,alpha,risk_mat)')
        dv.execute(' ex.set_omega(risk_mat)')
        results = dv.map(update, sol_list)
    
    print(results)
    
    for i in range(8) :
        for j in range(len(results[i][0])) :
            list_feasi_obj.append(results[i][0][j])
            list_feasi_sol.append(results[i][0][j])
    try:
        w_index = result.index(min(result))
        result2 = results[1][w_index]
        print(result2)

        w_pre = w_i_pre_sum(result2)
        feasi += 1
        list_feasi_obj.append(result)
        print("------objective value 변화-------")
        print(result)
        
    except:
        infeasi += 1

        
print("solve print : feasible 횟수 : ",feasi)
print("solve print : infeasible 횟수 : ",infeasi)
print("저장된 objective value")
print(list_feasi_obj)
print("저장된 solution")
print(list_feasi_sol)

ValueError: too many values to unpack (expected 11)